In [6]:
#Standard Header used on the projects

#first the major packages used for math and graphing
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
import scipy.special as sp
import math

#Custome graph format style sheet
plt.style.use('Prospectus.mplstyle')

#If being run by a seperate file, use the seperate file's graph format and saving paramaeters
#otherwise set what is needed
if not 'Saving' in locals():
    Saving = True
if not 'Titles' in locals():
    Titles = False
if not 'Ledgends' in locals():
    Ledgends = True
if not 'FFormat' in locals():
    FFormat = '.eps'
if not 'location' in locals():
    #save location.  First one is for running on home PC, second for running on the work laptop.  May need to make a global change
    location = 'E:\\Documents\\Dan\\Code\\FigsAndPlots\\FigsAndPlotsDocument\\Figures\\'
    #location = 'C:\\Users\\dhendrickson\\Documents\\github\\FigsAndPlots\\FigsAndPlotsDocument\\Figures\\'


#Standard cycle for collors and line styles
default_cycler = (cycler('color', ['0.00', '0.40', '0.60', '0.70']) + cycler(linestyle=['-', '--', ':', '-.']))
plt.rc('axes', prop_cycle=default_cycler)

In [7]:
class EFIT:
    __ts = 0
    __ds = 0

    def __init__(self, xGrid, yGrid, zGrid, dStep, tStep):
        #Initialize with the size of the grid in X, Y, and Z number of nodes.  The distance step, and the time step


        #Velocity grid with 3 part vector at each node point, for 4 time steps
        self.GridShapeV = (3,xGrid,yGrid,zGrid,4)
        #Stresses with 3 part vector in each of 3 part phaces, at each nod epoint, for 4 ime steps
        self.GridShapeS = (3,3,xGrid,yGrid,zGrid,4)
        #materials property gird.  Initially 3 properties needed: density, Lame 1, Lame 2
        self.GridShapeP = (xGrid,yGrid,zGrid,3)
     
        self.GridPoints = xGrid*yGrid*zGrid
        
        
        #define empty grid for the 3 directions of velocity for 4 time steps
        self.Gv = np.zeros(3*self.GridPoints*4,dtype="float32").reshape(*self.GridShapeV)
        #define empty grid for the 3 directions of stress on 3 dimmensions of faces for 4 time steps
        self.Gs = np.zeros(3*3*self.GridPoints*4,dtype="float32").reshape(*self.GridShapeS)
        #define empty grid for the 3 scalar material properties at each node point.  Can honly hold scalar properties
        #Assumed properties are density, Lame 1, Lame 2
        self.Gp = np.zeros(3*self.GridPoints,dtype="float32").reshape(*self.GridShapeP)
        
        self.__MaxX = xGrid
        self.__MaxY = yGrid
        self.__MaxZ = zGrid

        self.__ds = dStep
        self.__ts = tStep

    def CheckStressBoundary(self,x,y,z,Ds):
        #checks to see if a grid is at a boundary, and if so, adjusts boundary conditions appropriately
        #
        # Inputs: x,y,z coordinates of cube in question
        #
        # Outputs: Updated (if boundary) delta stress matrix

        #at front and back faces, stresses perpendicular to the face are 0:
        if x == 0 or x == self.__MaxX:
            Ds[0,:]=0
        
        #at top face, stresses perpendicular to the face are 0:
        if y == self.__MaxY:
            Ds[1,:]=0
        
        #at side faces, stresses perpendicular to the face are 0:
        if z == 0 or z == self.__MaxZ:
            Ds[2,:]=0
        
        return Ds

    def CheckVelocityBoundary(self,x,y,z,Dv):
        #checks to see if a grid is at a boundary, and if so, adjusts boundary conditions appropriately
        #
        # Inputs: x,y,z coordinates of cube in question
        #
        # Outputs: Updated (if boundary) delta velocity vector
        
        #lower boundary is fixed, velocity is Zero
        if y == 0:
            Dv[:]=0

        return Dv

    
    def DeltaStress(self,x,y,z):
        #Gets the change in the stresses per time at a certain coordinate juncture 
        #
        # Inputs: x,y,z coordinates of the cube in question.  Last time is assumed
        #
        # Outputs: 6 dimmensions of stress.  
        
        #Calculated stresses based on 3.55
        Ds = np.zeros((3,3))
        
        Ds[0,0] =  ((1/self.__ds) *
                    ((self.Gp[1,x,y,z]+2*self.Gp[2,x,y,z])*(self.Gv[0,x,y,z,3]-self.Gv[0,x-1,y,z,3]) +
                        self.Gp[1,x,y,z]*(self.Gv[1,x,y,z,3]-self.Gv[1,x,y-1,z,3]+self.Gv[2,x,y,z,3]-self.Gv[2,x,y,z-1,3])
                        )
                    )
        
        Ds[1,1] =  ((1/self.__ds) *
                    ((self.Gp[1,x,y,z]+2*self.Gp[2,x,y,z])*(self.Gv[1,x,y,z,3]-self.Gv[1,x,y-1,z,3]) +
                        self.Gp[1,x,y,z]*(self.Gv[0,x,y,z,3]-self.Gv[0,x-1,y,z,3]+self.Gv[2,x,y,z,3]-self.Gv[2,x,y,z-1,3])
                        )
                    )
        
        Ds[2,2] =  ((1/self.__ds) *
                    ((self.Gp[1,x,y,z]+2*self.Gp[2,x,y,z])*(self.Gv[2,x,y,z,3]-self.Gv[2,x,y,z-1,3]) +
                        self.Gp[1,x,y,z]*(self.Gv[0,x,y,z,3]-self.Gv[0,x-1,y,z,3]+self.Gv[1,x,y,z,3]-self.Gv[1,x,y-1,z,3])
                        )
                    )
        Ds[0,1] =  (
                    (1/self.__ds) *
                    (4/((1/self.Gp[2,x,y,z])+(1/self.Gp[2,x+1,y,z])+(1/self.Gp[2,x,y+1,z])+(1/self.Gp[2,x+1,y+1,z]))) *
                    (self.Gv[0,x,y+1,z,3]-self.Gv[0,x,y,z,3] +self.Gv[1,x+1,y,z,3]-self.Gv[1,x,y,z,3] )
                    )
        Ds[0,2] =  (
                    (1/self.__ds) *
                    (4/((1/self.Gp[2,x,y,z])+(1/self.Gp[2,x+1,y,z])+(1/self.Gp[2,x,y,z+1])+(1/self.Gp[2,x+1,y,z+1]))) *
                    (self.Gv[0,x,y,z+1,3]-self.Gv[0,x,y,z,3] +self.Gv[2,x+1,y,z,3]-self.Gv[2,x,y,z,3] )
                    )
        Ds[1,2] =  (
                    (1/self.__ds) *
                    (4/((1/self.Gp[2,x,y,z])+(1/self.Gp[2,x,y+1,z])+(1/self.Gp[2,x,y,z+1])+(1/self.Gp[2,x,y+1,z+1]))) *
                    (self.Gv[1,x,y,z+1,3]-self.Gv[1,x,y,z,3] +self.Gv[2,x,y+1,z,3]-self.Gv[2,x,y,z,3] )
                    )

        Ds = self.CheckStressBoundary(x,y,z,Ds)

        return Ds

    def DeltaVelocity(self, x,y,z):
        #Gets the change in the velocity per time at a certain coordinate juncture 
        #
        # Inputs: x,y,z coordinates of the cube in question.  Last time is assumed
        #
        # Outputs: 3 dimmensions of velocity.  
        
        #Calculated velocity based on 3.54

        DV = np.zeros(3)

        DV[0] = ((1 / self.__ds ) *
                              (2 / (self.Gp[0,x,y,z]+self.Gp[0,x+1,y,z])) *
                              (self.Gs[0,0,x+1,y,z,3] - self.Gs[0,0,x,y,z,3] + self.Gs[0,1,x,y,z,3] - self.Gs[0,1,x,y-1,z,3] + self.Gs[0,2,x,y,z,3] -self.Gs[0,2,x,y,z-1,3])
                              )
        #calculate velocity in y based on 3.54
        DV[1] = ((1 / self.__ds ) *
                              (2 / (self.Gp[0,x,y,z]+self.Gp[0,x,y+1,z])) *
                              (self.Gs[0,1,x,y,z,3] - self.Gs[0,1,x-1,y,z,3] + self.Gs[1,1,x,y+1,z,3] - self.Gs[1,1,x,y,z,3] + self.Gs[1,2,x,y,z,3] -self.Gs[1,2,x,y,z-1,3])
                              )
        #calculate velocity in z based on 3.54
        DV[2] = ((1 / self.__ds ) *
                              (2 / (self.Gp[0,x,y,z]+self.Gp[0,x,y,z+1])) *
                              (self.Gs[0,2,x,y,z,3] - self.Gs[0,2,x-1,y,z,3] + self.Gs[1,2,x,y,z,3] - self.Gs[1,2,x,y-1,z,3] + self.Gs[2,2,x,y,z+1,3] -self.Gs[2,2,x,y,z,3])
                              )
        
        DV = self.CheckVelocityBoundary(x,y,z,DV)

        return DV
    
    def UpdateStresses(self, x, y, z):
        #Updates velocity based off of previous velocities and delta velocity times time
        #
        # Inputs: Coordinates of cube in question.  Assumed last time step
        #
        # Output: updated self.Gs matrix

        for i in range(3):
            self.Gs[:,:,x,y,z,i] = self.Gs[:,:,x,y,z,i+1]
        
        delV = self.DeltaStress(x,y,z)

        for i in range(3):
                for j in range(3):
                    self.Gs[i,j,x,y,z,3] = self.Gv[i,j,x,y,z,2] + delV[i] * self.__ts

        return self

    def UpdateVelocity(self, x, y, z):
        #Updates velocity based off of previous velocities and delta velocity times time
        #
        # Inputs: Coordinates of cube in question.  Assumed last time step
        #
        # Output: updated self.Gs matrix

        for i in range(3):
            self.Gv[:,x,y,z,i] = self.Gv[:,x,y,z,i+1]
        
        delV = self.DeltaVelocity(x,y,z)

        for i in range(3):
            self.Gv[i,x,y,z,3] = self.Gv[i,x,y,z,2] + delV[i,j] * self.__ts

        return self

    def FrocingFunction(self, t):
        # Adds stresses from a force to the stress grid
        # Initially assumed a single force of a small plate sinosoidal ultrasound emitter.  More to be added later
        # 
        # Input is the time
        #
        # Outputs: no direct outputs, last time step stress is updated

        frequency = 10000
        EmitterPreasure = 1000
        
        
        EmitterWidth = 0.001 / self.__ds

        EmitterWidth = int(EmitterWidth)

        #emitter placed in middle of top face

        StartX = self.__MaxX / 2 - EmitterWidth / 2
        StartZ = self.__MaxZ / 2 - EmitterWidth / 2

        for i in range(EmitterWidth):
            for j in range(EmitterWidth):
                self.Gs[1,1,i+StartX,self.__MaxY,j+StartZ,4] = np.sin(frequency * t) * EmitterPreasure

        return self



In [8]:
# set Constants:
runtime = 0.5
PoissonRatio = 0.3
YoungModulus = 20 * (10**9)
mu = 80 * (10**9)         #First Lame Parameter
lmbda = 2 * mu * PoissonRatio / (1 - 2 * PoissonRatio)     #second Lame Parameter
rho = 7800       #density

#Calculate speed of longitudinal and transverse waves
cl = np.sqrt((lmbda + 2* mu)/rho)
ct = np.sqrt(mu/rho)

#Choose ferquency to be used for excitment
frequency = 10000

#calculate wave length
omegal = cl / frequency
omegat = ct / frequency

# about 1foot (0.3m) of just the web of 175lbs rail 
BeamLength = 0.3
BeamHeight = 0.0762
BeamWidth = 0.00381

Rail Properties:
https://railroadrails.com/railroad-rail-specification/
Using 175LBS Rail baseline

Steel Propoerties:
https://www.jsg.utexas.edu/tyzhu/files/Some-Useful-Numbers.pdf

relation between properties na dparameters:
https://en.wikipedia.org/wiki/Lam%C3%A9_parameters


In [9]:
#Set time step and grid step to be 10 steps per frequency and ten steps per wavelength respectively
ts = 1 / frequency / 10    #time step
gs = min(omegal, omegat) / 10    #grid step

Tsteps = int(math.ceil(runtime / ts)) + 1       #total Time STeps

gl = int(math.ceil(BeamLength / gs))        #number of grid points
gh = int(math.ceil(BeamHeight / gs))
gw = int(math.ceil(BeamLength / gs))


In [10]:
#Initialize EFIT Model
Rail = EFIT(gl, gh, gw, ts, gs)

#Set Material Properties consitant througout
Rail.Gp[0,:,:,:] = rho  #constant Density
Rail.Gp[1,:,:,:] = lmbda #Constant first Lamee parameter 
Rail.Gp[2,:,:,:] = mu  #constant second Lamee parameter


IEEE ultrasonics
SBIE NDE section NDE and smart structures
quantitiative

In [ ]:
# Initiallize Time step 1:




In [ ]:
#Run main function for time:
for i in range(Tsteps - 1):
    t = (i+1) * ts

    #Update stresses with forcing function
    Rail.FrocingFunction(t)

    #update Velocity:
    for x in range(gl):
        for y in range(gh):
            for z in range(gw):
                Rail.UpdateVelocity(x,y,z)
    
    #Update Stresses at next half step:
    for x in range(gl):
        for y in range(gh):
            for z in range(gw):
                Rail.UpdateStresses(x,y,z)

    